In [3]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from IPython import display



In [36]:

#@task(retries=3)
def ingest_data(dataset_url):
	df = pd.read_parquet(dataset_url, engine='pyarrow')
	return df

#@task(log_prints=True)
def transform_data(df):
	df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
	df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
	return df



In [ ]:
def etl_gcs_to_bq(year, month, color):

    path = extract_from_gcs(color, year, month)
    df = transform(path)
    numRowsProcessed = df.shape[0]
    print(f"Rows processed: {numRowsProcessed}")
    write_bq(df)


@flow()
def etl_parent_flow(color: str = "yellow", months: list[int] = [1,2], year: int = 2021):
    for month in months:
        etl_gcs_to_bq(year, month, color)
    

if __name__ == "__main__":
    color="yellow"
    months=[2,3]
    year=2019
    etl_parent_flow(color,months, year)

In [63]:
%%time


#@task(log_prints=True)
def export_data_gcs(path):
	gcs_block = GcsBucket.load("zoom-gcs")
	gcs_block.upload_from_path(from_path=path, to_path=path)

	return path


#@flow()
def etl_web_to_gcs(year, month, color):
	link_body = f"{color}_tripdata_{year}-{month:0>2}"
	print(link_body)
	dataset_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{link_body}.parquet"
	df = ingest_data(dataset_url)
	display(df.shape)
#	df_clean = transform_data(df)
#	path = export_data_local(df_clean, color, dataset_file)
#	export_data_gcs(path)


if __name__ == "__main__":
	color = "green"
	month = 4
	year = 2019
	etl_web_to_gcs(year, month, color)


green_tripdata_2019-04


(567852, 20)

CPU times: user 337 ms, sys: 175 ms, total: 512 ms
Wall time: 4.92 s


In [67]:
7049370+7866620

14915990

In [56]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2020-01.parquet', engine='pyarrow')

In [59]:
df.shape


(447770, 20)